# Create session-level measurements

This notebook concatenates all of the measurements collected on various dates into one csv per site. The metrics consist of the following measures, with one summary statistic per session-treatment group:

- indoor air temperature (mean, std dev, quartiles)
- indoor relative humidity (mean, std dev, quartiles)
- outdoor air temperature, avg/min/max over the date of each session.
- outdoor relative humidity, avg/min/max over the date of each session.
- participant level mean T and RH, averaged over duration of session.
- operative temp in control and tx
- CO2 in control and tx

This also aggregates the module timing data and saves it in a separate file.

These files are used for plotting in the separate `publication_figures_and_tables.ipynb` notebook

## Setup

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pickle
from datetime import timedelta
from os.path import basename
from pathlib import Path

import pandas as pd
import seaborn as sns
from utilities import (
    Settings,
    get_timing_df_berk,
    get_timing_df_bus,
    load_vals_berkeley,
    load_vals_bus,
)

sns.set()
sns.set_context("talk")

idx = pd.IndexSlice

In [4]:
home_dir_s = (
    "/Users/ianbolliger/Library/CloudStorage/Dropbox-Personal/Temperature & Behavior/"
    "Replication preparation"
)

In [5]:
home_dir = Path(home_dir_s)
s = Settings(home_dir)

In [6]:
def calc_session_level_indoor_vals(dfs, timing_df, site_settings):
    out_df = pd.DataFrame()
    idx = pd.IndexSlice

    for grp_ix, grp in enumerate(["control", "treatment"]):
        grp_df = timing_df[timing_df["Treatment group"] == grp_ix]

        # one dataframe for each sensor location
        loc_dfs = []
        for loc in (
            site_settings.sens_locs + [str(i) for i in range(1, 7)] + ["Top", "co2"]
        ):
            tmp_df = dfs[grp][loc]
            tmp_df["sess"] = pd.cut(tmp_df.index, bins=grp_df.index)
            grouped = tmp_df.groupby("sess", observed=True).describe()

            # drop pilot sessions where we don't have room-level sensors (only
            # individual)
            grouped = grouped[grouped.loc[:, idx[:, "count"]].sum(axis=1) != 0]

            # don't need # of measurements taken in session
            grouped = grouped.drop(columns="count", level=1)

            loc_dfs.append(grouped)

        # estimate average values for sessions where only one sensor recording
        # then average the sensors for each session
        room_sesh_vals = average_two_sensors(loc_dfs)

        # clarify that these are indoor temps and RH vals
        def renamer(x):
            if x == "one_sensor_only":
                return "one_sensor_only_in"
            return x.split("_")[0] + "_in_" + x.split("_")[1]

        sesh_vals = room_sesh_vals.rename(renamer, axis=1)

        # add participant-level sensors
        for s_ix in range(1, 7):
            this_df = loc_dfs[s_ix + 1]

            # convert to single-level index
            this_df.columns = [
                (j[0] + "_p{}_".format(s_ix) + j[1]).rstrip("_")
                for j in this_df.columns.values
            ]

            # join onto sesh_vals dataframes
            sesh_vals = sesh_vals.join(this_df, how="outer")

        # add operative temp
        this_df = loc_dfs[8]
        this_df.columns = [
            (j[0] + "_" + j[1]).rstrip("_") for j in this_df.columns.values
        ]
        sesh_vals = sesh_vals.join(this_df, how="outer")

        # add CO2
        this_df = loc_dfs[9]
        this_df.columns = [
            (j[0] + "_" + j[1]).rstrip("_") for j in this_df.columns.values
        ]
        sesh_vals = sesh_vals.join(this_df, how="outer")

        # join onto treatment group / session # data
        sesh_vals = sesh_vals.join(grp_df, how="inner")
        out_df = pd.concat((out_df, sesh_vals))

    # format nicely
    out_df = out_df.rename(
        columns={
            "Date": "date",
            "Session in day": "session",
            "Treatment group": "treatment",
        }
    )
    out_df = out_df.set_index(["date", "session", "treatment"])
    out_df = out_df.drop(columns=["start_time", "end_time"])
    out_df.columns = [c.rstrip("%") for c in out_df.columns]
    out_df = out_df.sort_index()
    out_df = out_df[out_df.notnull().any(axis=1)]

    return out_df


def average_two_sensors(loc_dfs):
    loc_dfs = bias_correct_one_sensor(loc_dfs)
    sesh_vals = (loc_dfs[0] + loc_dfs[1]) / 2
    sesh_vals["one_sensor_only"] = pd.DataFrame(
        [loc_dfs[0]["one_sensor_only"], loc_dfs[1]["one_sensor_only"]]
    ).max()
    return sesh_vals


def bias_correct_one_sensor(loc_dfs):
    """When only one sensor in a room, bias correct that sensor to estimate
    the average of the two sensors for that session."""
    # find mean difference in sensor
    mean_diff = (loc_dfs[1] - loc_dfs[0]).mean()
    # flatten index
    mean_diff.index = [(i[0] + "_" + i[1]).rstrip("_") for i in mean_diff.index.values]

    # get df with all sessions where at least one sensor was working
    all_locs = loc_dfs[1].join(loc_dfs[0], rsuffix="0", lsuffix="1")

    # adjust for one-sensor-only times
    # workaround b/c reindex fails with intervalIndex (bug)
    for i in [0, 1]:
        loc_dfs[i] = all_locs.loc[
            :, idx[[k for k in all_locs.columns.levels[0] if k[-1] == str(i)], :]
        ]

        # convert to single-level index
        loc_dfs[i] = loc_dfs[i].rename(lambda x: x[:-1], axis=1, level=0)
        loc_dfs[i].columns = [
            (j[0] + "_" + j[1]).rstrip("_") for j in loc_dfs[i].columns.values
        ]

        # mark where we did bias correction
        loc_dfs[i]["one_sensor_only"] = loc_dfs[i].isnull().any(axis=1)

    loc_dfs[1].iloc[:, :-1] = (
        loc_dfs[1]
        .iloc[:, :-1]
        .where(~loc_dfs[1]["one_sensor_only"], loc_dfs[0].iloc[:, :-1] + mean_diff)
    )
    loc_dfs[0].iloc[:, :-1] = (
        loc_dfs[0]
        .iloc[:, :-1]
        .where(~loc_dfs[0]["one_sensor_only"], loc_dfs[1].iloc[:, :-1] - mean_diff)
    )

    return loc_dfs


def add_outdoor_vals(output_df, dfs_outdoor, timing_df):
    res = output_df.copy()
    for i in ["min", "mean", "max"]:
        to_join = dfs_outdoor[i].copy()
        to_join.index = pd.to_datetime(to_join.index)
        to_join = (
            timing_df.join(to_join, on="Date", how="inner")
            .drop_duplicates()
            .set_index(["Date", "Session in day", "Treatment group"])
            .loc[:, ["T", "RH"]]
        )
        to_join.columns = [r + "_out_daily" + i for r in to_join.columns]
        to_join.index.names = res.index.names
        res = res.join(to_join, how="outer")
    return res

## Berkeley
### Load Data

In [6]:
# load timing data
timing_df_berk = get_timing_df_berk(s)

# load all raw data
dfs_berk = load_vals_berkeley(s)

# save aggregated raw data for future loading
with (s.clean_data_dir / "sensor_data_berk.pickle").open("wb") as f:
    pickle.dump(dfs_berk, f)

# get session-level aggregate statistics
out_df_berk = calc_session_level_indoor_vals(dfs_berk["indoor"], timing_df_berk, s.berk)
out_df_berk = add_outdoor_vals(out_df_berk, dfs_berk["outdoor"], timing_df_berk)

# save aggregate data
out_df_berk.to_csv(s.berk.env_out_path)

## Busara

In [7]:
# load timing dataframe
timing_df_bus = get_timing_df_bus(s)

# load values dataframe
dfs_bus = load_vals_bus(s)

# save raw data for future loading
with (s.clean_data_dir / "sensor_data_bus.pickle").open("wb") as f:
    pickle.dump(dfs_bus, f)

# get session-level aggregate statistics
out_df_bus = calc_session_level_indoor_vals(dfs_bus["indoor"], timing_df_bus, s.bus)
out_df_bus = add_outdoor_vals(out_df_bus, dfs_bus["outdoor"], timing_df_bus)

# save aggregate data
out_df_bus.to_csv(s.bus.env_out_path)

## Module Timing Table

In [8]:
# Get dataframes of T timeseries by session

ts_dfs = {}

in_dfs = {"California": dfs_berk["indoor"], "Nairobi": dfs_bus["indoor"]}
timing_dfs = {"California": timing_df_berk, "Nairobi": timing_df_bus}
site_settings = {"California": s.berk, "Nairobi": s.bus}
for site in s.sites:
    ts_dfs[site] = {}
    timing_df = timing_dfs[site]

    for grp_ix, grp in enumerate(s.txs):
        grp_df = timing_df[timing_df["Treatment group"] == grp_ix]
        ts_dfs[site][grp] = {}

        # one dataframe for each sensor location
        loc_dfs = []
        for loc in (
            site_settings[site].sens_locs
            + [str(i) for i in range(1, 7)]
            + ["Top", "co2"]
        ):
            tmp_df = in_dfs[site][grp][loc].reset_index()

            # assign to session
            tmp_df["sess"] = pd.cut(tmp_df["time"], bins=grp_df.index)

            # keep only measurements during sessions
            tmp_df = tmp_df[tmp_df["sess"].notnull()]

            # get interval for each observation
            tmp_df["t_ix_sess"] = tmp_df.groupby("sess", observed=True).cumcount()

            # adjust for when we had 5 min intervals in berkeley
            if loc in site_settings[site].sens_locs:
                tmp_df["t_ix_sess"] = tmp_df["t_ix_sess"].where(
                    tmp_df["time"] >= s.berk.sensor_swap_date, tmp_df["t_ix_sess"] * 5
                )

            # pivot to wide table
            if loc == "Top":
                this_val = ["Top"]
            elif loc == "co2":
                this_val = ["co2"]
            else:
                this_val = ["T", "RH"]
            for t in this_val:
                val_by_sess = tmp_df.pivot(
                    index="sess", columns="t_ix_sess", values=t
                ).T
                loc_dfs.append(val_by_sess)

        # ignoring small # of sessions where only one sensor was in room
        combined_cols = loc_dfs[0].columns.intersection(loc_dfs[2].columns)
        mean_df = (
            loc_dfs[0].loc[:, combined_cols] + loc_dfs[2].loc[:, combined_cols]
        ) / 2
        ts_dfs[site][grp]["room_avg_T"] = mean_df
        mean_df_RH = (
            loc_dfs[1].loc[:, combined_cols] + loc_dfs[3].loc[:, combined_cols]
        ) / 2
        ts_dfs[site][grp]["room_avg_RH"] = mean_df_RH
        for i in range(1, 7):
            ts_dfs[site][grp]["p{}_T".format(i)] = loc_dfs[2 * (i + 1)]
            ts_dfs[site][grp]["p{}_RH".format(i)] = loc_dfs[2 * (i + 1) + 1]
        ts_dfs[site][grp]["T_op"] = loc_dfs[16]
        ts_dfs[site][grp]["CO2"] = loc_dfs[17]

with s.ts_dfs_path.open("wb") as f:
    pickle.dump(ts_dfs, f)

In [9]:
modules = [
    "production",
    "dictator",
    "risk_game",
    "t_preference",
    "trust",
    "public_goods",
    "ravens",
    "joy_O_D",
    "survey",
    "charity",
]

tzs = {"California": "US/Pacific", "Nairobi": "Africa/Nairobi"}


def get_mean_temp(row, temps_df, st_time, loc, tx):
    st = int((row["start_time"] - st_time).total_seconds() / 60)
    end = int((row["end_time"] - st_time).total_seconds() / 60)

    if st < 0:
        # There are many cases in Nairobi where the start time for production
        # module occurs before the recorded starting time in the experiment_timing.csv
        # file. IB is investigating this w/ Ray but the hunch is that the Busara team
        # opened the experiment to the introduction page prior to participants entering
        # room. This is b/c the time spent on the intro page is often very long. So we
        # are assuming the start times are 0 for these cases.
        st = 0
    return pd.Series([temps_df[st:end].mean(), st, end], index=["temps", "st", "end"])


module_temps = {
    "California": {
        "control": pd.DataFrame(index=modules),
        "treatment": pd.DataFrame(index=modules),
    },
    "Nairobi": {
        "control": pd.DataFrame(index=modules),
        "treatment": pd.DataFrame(index=modules),
    },
}
start_times = {
    "California": {
        "control": pd.DataFrame(index=modules),
        "treatment": pd.DataFrame(index=modules),
    },
    "Nairobi": {
        "control": pd.DataFrame(index=modules),
        "treatment": pd.DataFrame(index=modules),
    },
}
end_times = {
    "California": {
        "control": pd.DataFrame(index=modules),
        "treatment": pd.DataFrame(index=modules),
    },
    "Nairobi": {
        "control": pd.DataFrame(index=modules),
        "treatment": pd.DataFrame(index=modules),
    },
}


for loc in s.sites:
    date_dirs = getattr(s, s.site_map[loc]).modules_dir.glob("[!.]*")
    for d in date_dirs:
        sessions = d.glob("[!.]*")
        for se in sessions:
            sess = basename(se).split()
            if sess[2] == "C":
                tx = "control"
            elif sess[2] == "T":
                tx = "treatment"
            else:
                raise ValueError

            ts_fpath = list(se.glob("TimeSpent*"))
            if len(ts_fpath):
                ts_fpath = ts_fpath[0]
            else:
                print("No module time data: {}".format(se))
                continue

            this_df = pd.read_csv(ts_fpath, index_col="time_stamp")
            this_df.index = pd.to_datetime(this_df.index, unit="s")
            this_df = (
                this_df.tz_localize("UTC")
                .tz_convert(tzs[loc])
                .tz_localize(None)
                .reset_index()
            )

            this_df = this_df.loc[
                this_df["app_name"].isin(modules),
                [
                    "app_name",
                    "participant__id_in_session",
                    "time_stamp",
                    "seconds_on_page",
                ],
            ]
            this_df = this_df.sort_values(["participant__id_in_session", "app_name"])
            midtime = this_df.sort_values("time_stamp").iloc[
                int(this_df.shape[0] / 2), 2
            ]
            groups = this_df.groupby(["participant__id_in_session", "app_name"])

            # start times
            st = groups.first()
            # For production, take starting point as "end of intro slide"
            # b/c this comes first and in busara they navigated to this page
            # before the participants entered
            page_start = st["time_stamp"] - st["seconds_on_page"].apply(
                lambda x: timedelta(seconds=x)
            )
            st = page_start.where(
                st.index.get_level_values(1) != "production", st["time_stamp"]
            )
            st.name = "start_time"

            # end times
            end = groups.last()["time_stamp"]
            end.name = "end_time"

            # get times for temp data
            try:
                this_temps = ts_dfs[loc][tx]["room_avg_T"][midtime].interpolate()
                sess_st_time = this_temps.name.left
            except KeyError:
                continue

            this_times = pd.DataFrame([st, end]).T
            vals = this_times.apply(
                lambda x: get_mean_temp(x, this_temps, sess_st_time, loc, tx), axis=1
            )
            vals = vals[vals["temps"].notnull()]

            # average accross participants
            vals = vals.groupby(level=1).median()
            module_temps[loc][tx][this_temps.name] = vals["temps"]
            start_times[loc][tx][this_temps.name] = vals["st"]
            end_times[loc][tx][this_temps.name] = vals["end"]

No module time data: /Users/ianbolliger/Library/CloudStorage/Dropbox-Personal/Temperature & Behavior/Replication preparation/data/main/raw/xlab/modules/9-25/Session 2 T


In [10]:
idx = pd.IndexSlice
ix = pd.MultiIndex.from_product(
    [s.sites, s.txs, modules], names=["location", "group", "module"]
)
t_mod_df = pd.DataFrame(
    index=ix, columns=["st_time", "end_time", "mean_temp"]
).sort_index()
for loc in s.sites:
    for tx in s.txs:
        this_df = pd.DataFrame(
            {
                "st_time": start_times[loc][tx].median(axis=1),
                "end_time": end_times[loc][tx].median(axis=1),
                "mean_temp": module_temps[loc][tx].median(axis=1),
            }
        )
        this_df.index = pd.MultiIndex.from_product(
            [[loc], [tx], list(start_times[loc][tx].index)],
            names=["location", "group", "module"],
        )
        t_mod_df.loc[idx[loc, tx, :], :] = this_df

In [11]:
t_mod_df.to_parquet(s.module_temps_path)